Lucas Santos Nunez <br>
**IBM Captstone Project**


## Table of Contents



<font size = 3>

1.  <a href="#item1">Importing Packages</a>

2.  <a href="#item2">Scraping the Wikipedia Page</a>

3.  <a href="#item3">Pre-processing the data</a>

4.  <a href="#item4">Creating Map using the Folium Package</a>

5.  <a href="#item5">One Hot encoding for categorical for Venue Category</a>

6.  <a href="#item6">K MEANS of the Toronto Dataset</a>

7. <a href="#item7">Cluster Analysis </a>
    
    
 </font>



<a id='item1'></a>

## 1. Importing packages

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

<a id='item2'></a>

## 2. Scraping the wikipedia page

In [2]:
url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(url, 'lxml')

<a id='item3'></a>

## 3. Pre-processing the data

- Find the table and all rows
- Create tree lists to append the postalcode,borough and neighborhood
- Replace line breaks \n with null character, then get info that is in the text portion of the row
- Append all reuslts to the tree lists created
- In neigh final replace / for ,
- Create df_tronto from the three lists that appended th results on the for loop

In [3]:
table= soup.find("table").find_all("td")
postalcode=[]
borough=[]
neigh=[]
for i in range(0,len(table)):
    table_row = str(table[i].text).replace("\n","")
    postalcode_row = table_row[0:3]
    info_row = str(table[i].text).replace("\n","")[3:len(str(table[i].text))].replace(")","").split("(")
    borough_row = info_row[0]
    postalcode.append(postalcode_row)
    borough.append(borough_row)
    if len(info_row)!= 1:
        neigh.append(info_row[1])
    else:
        neigh.append(borough[i])

        
neigh_final = []        
for item in neigh:
    neigh_final.append(item.replace("/",","))
                
df_toronto = pd.DataFrame({"Postal Code":postalcode,"Borough":borough,"Neighborhood":neigh_final})                

Load the csv with the cordinates

In [4]:
coord_df= pd.read_csv(r"C:\Users\Lucas Santos Nunez\Downloads\Geospatial_Coordinates.csv")

In order to get the cordinates to the correspoding postal code, I used a inner join to do so

In [5]:
df_toronto_final = pd.merge(df_toronto, coord_df, how="inner", on=["Postal Code", "Postal Code"])

Remove the Not Assigned Borough's

In [6]:
df_toronto_final = df_toronto_final[df_toronto_final["Borough"]!= "Not Assigned"]

Glimpse of the final data frame

In [7]:
df_toronto_final.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


Daframe Info

In [8]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_toronto_final['Borough'].unique()),
        df_toronto_final.shape[0]
    )
)

The dataframe has 15 boroughs and 103 neighborhoods.


<a id='item4'></a>

## 4. Creating Map using the Folium Package

From the <a href="url">https://www.latlong.net/place/toronto-on-canada-27230.html</a>,I got the cordinates of Toronto City and used it to create a map using the folium package.


In [9]:
latitude = 43.651070
longitude = -79.347015
map_toronto = folium.Map(location=[latitude,longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto_final['Latitude'], df_toronto_final['Longitude'], df_toronto_final['Borough'], df_toronto_final['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='orange',
        fill=True,
        fill_color='black',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto) 

Showing the map

In [10]:
map_toronto

Including my foursquare credentials

In [11]:
CLIENT_ID = '3ANAALUQEQG10MN3B1LDX3JL5JFJM2OOOL5EGASNVKPALPMZ' # your Foursquare ID
CLIENT_SECRET = '3UZIQPDUQNZABLHKYJMN4XP0LG3O51VSPHE20BB5IZQ0UR2B' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

Pre built function in the IBM - NY example

In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Using the pre built function that for every neighborhood get venues Latitude Longitude and Category:


In [13]:
toronto_venues = getNearbyVenues(names=df_toronto_final['Neighborhood'],
                                   latitudes=df_toronto_final['Latitude'],
                                   longitudes=df_toronto_final['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park , Harbourfront
Lawrence Manor , Lawrence Heights
Ontario Provincial Government
Islington Avenue
Malvern , Rouge
Don MillsNorth
Parkview Hill , Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park , Princess Gardens , Martin Grove , Islington , Cloverdale
Rouge Hill , Port Union , Highland Creek
Don MillsSouth
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate , Bloordale Gardens , Old Burnhamthorpe , Markland Wood
Guildwood , Morningside , West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor , Wilson Heights , Downsview North
Thorncliffe Park
Richmond , Adelaide , King
Dufferin , Dovercourt Village
Scarborough Village
Fairview , Henry Farm , Oriole
Northwood Park , York University
The Danforth  East
Harbourfront East , Union Station , Toronto Islands
Little Portugal , Trinity
Kennedy Park , Ionview , East Birchmount Park
Bayview Village
D

Glimpse of the final DataFrame and saving it to csv for posterior use.

In [14]:
toronto_venues.head()
toronto_venues.to_csv(r"C:\Users\Lucas Santos Nunez\Downloads\toronto_venues.csv")

Quantity of Unique Categories

In [15]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 263 uniques categories.


<a id='item5'></a>

## 5. One Hot encoding for categorical for Venue Category

In [16]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,"Alderwood , Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor , Wilson Heights , Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park , Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,WillowdaleSouth,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.035714,0.0,0.0,0.0,0.0,0.0
96,WillowdaleWest,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
97,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
98,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


Dimension of the Dataframe

In [18]:
toronto_grouped.shape

(100, 263)

Usingthe pre built function of IBM NY Example to get the most present category of venue in every neighborhood

In [19]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Rank the top 10 venues of every neighborhood and then put the information into a Dataframe

In [20]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
toronto_venues_sorted = pd.DataFrame(columns=columns)
toronto_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    toronto_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

toronto_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Breakfast Spot,Skating Rink,Clothing Store,Latin American Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
1,"Alderwood , Long Branch",Pizza Place,Coffee Shop,Sandwich Place,Dance Studio,Pub,Gym,Airport Food Court,Deli / Bodega,Escape Room,Electronics Store
2,"Bathurst Manor , Wilson Heights , Downsview North",Coffee Shop,Bank,Park,Pizza Place,Shopping Mall,Bridal Shop,Sandwich Place,Diner,Sushi Restaurant,Mobile Phone Shop
3,Bayview Village,Café,Japanese Restaurant,Bank,Chinese Restaurant,Women's Store,Department Store,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
4,"Bedford Park , Lawrence Manor East",Restaurant,Coffee Shop,Sandwich Place,Cupcake Shop,Indian Restaurant,Butcher,Café,Italian Restaurant,Sushi Restaurant,Liquor Store


<a id='item6'></a>

## 6. K MEANS of the Toronto Dataset

Applying the K Means algorithm to the Toronto Dataset

In [21]:
kclusters = 5

toronto_grouped_clustering =toronto_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_[0:10] 

array([2, 4, 2, 2, 2, 2, 2, 2, 2, 0])

Adding the Kmeans Label as a Column then merging it on Toronto initial(df_toronto_final) to get latitude and longitude of every neighborhood 

In [22]:
toronto_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto_final

toronto_merged = toronto_merged.join(toronto_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged = toronto_merged.dropna()

toronto_merged["Cluster Labels"]= toronto_merged["Cluster Labels"].apply(lambda x:int(x)) # check the last columns!

Creating the map to display the categorized neighborhood using matplotlib colormap.

In [23]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    cluster = int(cluster)
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item7'></a>

## 7. Cluster Analysis
Creating a function to return the dataframe of one cluster that will contain:<br>

- Postal Code<br>

- Neighboorhood<br>

- 1st Most Common Venue<br>	

- 2nd Most Common Venue<br>

- 3rd Most Common Venue<br> 	

- 4th Most Common Venue<br> 	

- 5th Most Common Venue<br> 	

- 6th Most Common Venue<br>	

- 7th Most Common Venue<br>	

- 8th Most Common Venue<br>	

- 9th Most Common Venue<br>

- 10th Most Common Venue<br>

In [24]:
def cluster_analysis(i):
    df = toronto_merged[toronto_merged['Cluster Labels'] == i]
    df = df[["Postal Code",
            "Neighborhood",
            "1st Most Common Venue",
            "2nd Most Common Venue",
            "3rd Most Common Venue",
            "4th Most Common Venue", 
            "5th Most Common Venue", 
            "6th Most Common Venue", 
            "7th Most Common Venue", 
            "8th Most Common Venue", 
            "9th Most Common Venue", 
            "10th Most Common Venue"]]
    return df

### Cluster 0

In [25]:
cluster_analysis(0)

,Postal Code,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,M6E,Caledonia-Fairbanks,Park,Women's Store,Pool,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant,Deli / Bodega
35,M4J,The Danforth East,Park,Intersection,Convenience Store,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Deli / Bodega
40,M3K,DownsviewEast,Park,Snack Place,Airport,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Donut Shop
52,M2M,"Willowdale , Newtonbrook",Park,Event Space,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
61,M4N,Lawrence Park,Park,Bus Line,Swim School,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Ethiopian Restaurant
64,M9N,Weston,Park,Jewelry Store,Convenience Store,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Deli / Bodega
66,M2P,York Mills West,Park,Moving Target,Convenience Store,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Deli / Bodega
68,M5P,Forest Hill North & West,Park,Trail,Sushi Restaurant,Jewelry Store,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Dance Studio
77,M9R,"Kingsview Village , St. Phillips , Martin Grov...",Park,Sandwich Place,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Women's Store,Deli / Bodega
85,M1V,"Milliken , Agincourt North , Steeles East , L'...",Park,Playground,Intersection,Electronics Store,Eastern European Restaurant,Drugstore,Escape Room,Donut Shop,Deli / Bodega,Dog Run


<p style="color:red">Conclusion</p>

As we can see, this cluster is based on the 1st Most Common Avenue,almost all Neighborhood included in this cluster are because they have parks as the most frequent avenue

### Cluster 1

In [26]:
cluster_analysis(1)

,Postal Code,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,M1J,Scarborough Village,Playground,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant


<p style="color:red">Conclusion</p>

This cluster contains only one neighbhorhood, probably because the most common venue that is not found on other normally are Palyground and Dog Run's

### Cluster 2

In [27]:
cluster_analysis(2)

,Postal Code,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M4A,Victoria Village,Hockey Arena,Coffee Shop,Portuguese Restaurant,Intersection,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
2,M5A,"Regent Park , Harbourfront",Coffee Shop,Park,Bakery,Café,Pub,Greek Restaurant,Sandwich Place,Bank,Spa,Beer Store
3,M6A,"Lawrence Manor , Lawrence Heights",Clothing Store,Furniture / Home Store,Accessories Store,Miscellaneous Shop,Vietnamese Restaurant,Coffee Shop,Boutique,Distribution Center,Dessert Shop,Dim Sum Restaurant
4,M7A,Ontario Provincial Government,Coffee Shop,Sushi Restaurant,Burrito Place,Yoga Studio,Theater,Sandwich Place,Salad Place,Diner,Burger Joint,Japanese Restaurant
7,M3B,Don MillsNorth,Japanese Restaurant,Gym,Café,Caribbean Restaurant,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Women's Store
...,...,...,...,...,...,...,...,...,...,...,...,...
96,M4X,"St. James Town , Cabbagetown",Pizza Place,Coffee Shop,Pharmacy,Chinese Restaurant,Bakery,Restaurant,Café,Pub,Italian Restaurant,Bank
97,M5X,"First Canadian Place , Underground city",Coffee Shop,Sandwich Place,Café,Hotel,Bank,Japanese Restaurant,Asian Restaurant,Gym,Restaurant,Sushi Restaurant
99,M4Y,Church and Wellesley,Sushi Restaurant,Japanese Restaurant,Coffee Shop,Restaurant,Indian Restaurant,Men's Store,Mediterranean Restaurant,Burrito Place,Gay Bar,Fast Food Restaurant
100,M7Y,Enclave of M4L,Light Rail Station,Yoga Studio,Auto Workshop,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Comic Shop,Pizza Place,Restaurant


<p style="color:red">Conclusion</p>

As we can see, this cluster is based on food venues,almost all Neighborhood included in this cluster are because they have Cofee Shop,Café and Bakery as the 1st,2nd,3rd most common venues 

### Cluster 3

In [28]:
cluster_analysis(3)

,Postal Code,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
53,M3M,DownsviewCentral,Baseball Field,Food Truck,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Women's Store,Falafel Restaurant
57,M9M,"Humberlea , Emery",Baseball Field,Furniture / Home Store,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Falafel Restaurant,Doner Restaurant
101,M8Y,"Old Mill South , King's Mill Park , Sunnylea ,...",Baseball Field,Women's Store,Department Store,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant


<p style="color:red">Conclusion</p>

As we can see, this cluster is based on the 1st Most Common Avenue,almost all Neighborhood included in this cluster are because they have Baseball Field as the most frequent avenue

### Cluster 4

In [29]:
cluster_analysis(4)

,Postal Code,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,Parkwoods,Park,Fast Food Restaurant,Food & Drink Shop,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run
6,M1B,"Malvern , Rouge",Fast Food Restaurant,Women's Store,Department Store,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant
27,M2H,Hillcrest Village,Pool,Mediterranean Restaurant,Fast Food Restaurant,Golf Course,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
47,M4L,"India Bazaar , The Beaches West",Fast Food Restaurant,Board Shop,Pizza Place,Brewery,Sandwich Place,Burrito Place,Steakhouse,Restaurant,Italian Restaurant,Ice Cream Shop
63,M6N,"Runnymede , The Junction North",Brewery,Bus Line,Pizza Place,Convenience Store,Women's Store,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
72,M2R,WillowdaleWest,Coffee Shop,Grocery Store,Pharmacy,Pizza Place,Supermarket,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
82,M1T,"Clarks Corners , Tam O'Shanter , Sullivan",Pizza Place,Chinese Restaurant,Gas Station,Coffee Shop,Thai Restaurant,Fast Food Restaurant,Bank,Italian Restaurant,Dessert Shop,Dim Sum Restaurant
88,M8V,"New Toronto , Mimico South , Humber Bay Shores",Liquor Store,Pizza Place,Seafood Restaurant,Fast Food Restaurant,Mexican Restaurant,Flower Shop,Restaurant,American Restaurant,Café,Pharmacy
89,M9V,"South Steeles , Silverstone , Humbergate , Jam...",Grocery Store,Beer Store,Coffee Shop,Video Store,Sandwich Place,Pharmacy,Pizza Place,Fast Food Restaurant,Distribution Center,Dessert Shop
90,M1W,"Steeles West , L'Amoreaux West",Fast Food Restaurant,Electronics Store,Coffee Shop,Bank,Pharmacy,Pizza Place,Breakfast Spot,Supermarket,Chinese Restaurant,Sandwich Place


<p style="color:red">Conclusion</p>

As we can see, this cluster is based on food and drink venues,almost all Neighborhood included in this cluster are because they have Fast Food Reaustants, Beet Strotre, Liquor Store and Deli/Bodega. 